In [1]:
import pytorch_lightning as pl
import torch 
from pytorch_lightning.metrics.functional.classification import accuracy
from tqdm import tqdm
from src import DataModule, Resnet
import torchvision
import pandas as pd 
from pathlib import Path
from skimage import io

In [3]:
checkpoint = torch.load('./resnet18-512-val_acc=0.85654.ckpt')
hparams = checkpoint['hyper_parameters']
hparams

{'lr': 0.0003,
 'optimizer': 'Adam',
 'batch_size': 256,
 'max_epochs': 50,
 'precision': 16,
 'subset': 0,
 'test_size': 0.2,
 'seed': 42,
 'size': 512,
 'backbone': 'resnet18'}

In [3]:
dm = DataModule(
    file = 'train_extra.csv', 
    batch_size=64
)
dm.setup()

Training samples:  21642
Validation samples:  5411


In [ ]:
model = Resnet.load_from_checkpoint(checkpoint_path='./resnet18-256-val_acc=0.78398.ckpt')
model.hparams

In [ ]:
 def evaluate(model, dl):   
    model.eval()
    model.cuda()
    acc = []
    with torch.no_grad():
        t = tqdm(dl)
        for x, y in t:
            x, y = x.cuda(), y.cuda()
            y_hat = model(x)
            acc.append(accuracy(y_hat, y).item())
            t.set_description(f"acc {np.mean(acc):.5f}")
            
evaluate(model, dm.val_dataloader())

In [ ]:
class Preprocess(torch.nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self, x):
        x = x.float() / 255.
        x = x.permute(0, 3, 1, 2)
        return x 
    
class Postprocess(torch.nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self, x):
        return torch.argmax(x, dim=1)

In [ ]:
script = torch.jit.script(torch.nn.Sequential(
    Preprocess(),
    model.resnet.cpu(),
    Postprocess()
))
torch.jit.save(script, "model.pt")

In [ ]:
 def evaluate2(model, dl):   
    model.eval()
    model.cuda()
    acc = []
    with torch.no_grad():
        t = tqdm(dl)
        for x, y in t:
            x, y = x.cuda(), y.cuda()
            # simulate test
            x *= 255. 
            x = x.permute(0, 2, 3, 1).long()
            #print(x.shape, x.dtype, x.max(), x.min())
            y_hat = model(x)
            acc.append(accuracy(y_hat, y).item())
            t.set_description(f"acc {np.mean(acc):.5f}")

In [ ]:
loaded = torch.jit.load('model.pt')
evaluate2(loaded, dm.val_dataloader())

In [ ]:
path = Path('./data/test_images')
images = os.listdir(path)
images_paths = [str(path/img) for img in images]
len(images)

In [ ]:
def predict(model, images, bs=32):   
    model.eval()
    model.cuda()
    preds = torch.tensor([]).cuda()
    batches = len(images) // bs + 1
    print(batches)
    with torch.no_grad():
        for b in tqdm(range(batches)):
            imgs = images[bs*b:bs*(b+1)]
            imgs = torch.from_numpy(np.array([io.imread(img) for img in imgs]))
            y_hat = model(imgs.cuda())
            preds = torch.cat([preds, y_hat])
    return preds.long().cpu().numpy()

In [ ]:
loaded = torch.jit.load('model.pt')
preds = predict(loaded, images_paths)
preds

In [ ]:
submission = pd.DataFrame({'image_id': images, 'label': preds })
submission

In [ ]:
submission.to_csv('submission.csv', index=False)